
# Weather Chatbot with OpenWeather API

This notebook presents the implementation of a weather chatbot utilizing the OpenWeather API. The chatbot is designed to respond to user queries about current weather conditions such as humidity, temperature, and wind speed in various cities. The architecture leverages natural language processing (NLP) to classify user intents, with a machine learning model trained on weather-related data. Once an intent is recognized, the chatbot fetches real-time weather information from the OpenWeather API and formulates a response. The notebook walks through data preprocessing, model training, and interaction simulation.


In [64]:

# !pip install numpy nltk tensorflow datasets requests


In [65]:

import numpy as np
import nltk
import tensorflow as tf
import random
import json
import pickle
import requests
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from datasets import load_dataset

nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /Users/m1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/m1/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [66]:
import json

# Define the list of new intents and corresponding sentences
new_intents = [
    {
        "user_input": "Is it raining right now in New York?",
        "intent_extraction": {
            "entities": {
                "city": "New York"
            },
            "intent": "current_rain"
        }
    },
    {
        "user_input": "Will it rain tomorrow in Paris?",
        "intent_extraction": {
            "entities": {
                "city": "Paris",
                "date": "2024-06-15"
            },
            "intent": "forecast_rain"
        }
    },
    {
        "user_input": "What's the current humidity level in Tokyo?",
        "intent_extraction": {
            "entities": {
                "city": "Tokyo"
            },
            "intent": "current_humidity"
        }
    },
    {
        "user_input": "What will the humidity be like in London tomorrow?",
        "intent_extraction": {
            "entities": {
                "city": "London",
                "date": "2024-06-15"
            },
            "intent": "forecast_humidity"
        }
    },
    
    {
        "user_input": "What's the current humidity in San Francisco?",
        "intent_extraction": {
            "entities": {
                "city": "San Francisco"
            },
            "intent": "current_humidity"
        }
    },
    {
        "user_input": "How humid is it right now in Tokyo?",
        "intent_extraction": {
            "entities": {
                "city": "Tokyo"
            },
            "intent": "current_humidity"
        }
    },
    {
        "user_input": "Tell me the current humidity level in New York.",
        "intent_extraction": {
            "entities": {
                "city": "New York"
            },
            "intent": "current_humidity"
        }
    },
    {
        "user_input": "What is the current humidity in Paris?",
        "intent_extraction": {
            "entities": {
                "city": "Paris"
            },
            "intent": "current_humidity"
        }
    },
    {
        "user_input": "How humid is it in Berlin right now?",
        "intent_extraction": {
            "entities": {
                "city": "Berlin"
            },
            "intent": "current_humidity"
        }
    },
    {
        "user_input": "What's the humidity level in Sydney at the moment?",
        "intent_extraction": {
            "entities": {
                "city": "Sydney"
            },
            "intent": "current_humidity"
        }
    },
    {
        "user_input": "Current humidity in Rome, please.",
        "intent_extraction": {
            "entities": {
                "city": "Rome"
            },
            "intent": "current_humidity"
        }
    },
    {
        "user_input": "Check the current humidity in London.",
        "intent_extraction": {
            "entities": {
                "city": "London"
            },
            "intent": "current_humidity"
        }
    },
    {
        "user_input": "What's the humidity like in Bangkok now?",
        "intent_extraction": {
            "entities": {
                "city": "Bangkok"
            },
            "intent": "current_humidity"
        }
    },
    {
        "user_input": "Tell me the humidity level in Singapore right now.",
        "intent_extraction": {
            "entities": {
                "city": "Singapore"
            },
            "intent": "current_humidity"
        }
    },
    {
        "user_input": "How humid is it currently in Hong Kong?",
        "intent_extraction": {
            "entities": {
                "city": "Hong Kong"
            },
            "intent": "current_humidity"
        }
    },
    {
        "user_input": "Current humidity in Cape Town?",
        "intent_extraction": {
            "entities": {
                "city": "Cape Town"
            },
            "intent": "current_humidity"
        }
    },
    {
        "user_input": "What's the humidity in Vienna today?",
        "intent_extraction": {
            "entities": {
                "city": "Vienna"
            },
            "intent": "current_humidity"
        }
    },
    {
        "user_input": "How humid is it in Athens right now?",
        "intent_extraction": {
            "entities": {
                "city": "Athens"
            },
            "intent": "current_humidity"
        }
    },
    {
        "user_input": "Can you tell me the humidity level in Chicago?",
        "intent_extraction": {
            "entities": {
                "city": "Chicago"
            },
            "intent": "current_humidity"
        }
    },
    {
        "user_input": "What will the humidity be like in Tokyo tomorrow?",
        "intent_extraction": {
            "entities": {
                "city": "Tokyo",
                "date": "2024-06-15"
            },
            "intent": "forecast_humidity"
        }
    },
    {
        "user_input": "Can you tell me the humidity forecast for Paris next week?",
        "intent_extraction": {
            "entities": {
                "city": "Paris",
                "date": "2024-06-19"
            },
            "intent": "forecast_humidity"
        }
    },
    {
        "user_input": "What's the humidity forecast in Berlin for the weekend?",
        "intent_extraction": {
            "entities": {
                "city": "Berlin",
                "date": "2024-06-17",
                "time_of_day": "weekend"
            },
            "intent": "forecast_humidity"
        }
    },
    {
        "user_input": "Tell me about the humidity forecast in New York for tomorrow.",
        "intent_extraction": {
            "entities": {
                "city": "New York",
                "date": "2024-06-16"
            },
            "intent": "forecast_humidity"
        }
    },
    {
        "user_input": "What is the expected humidity level in Sydney this evening?",
        "intent_extraction": {
            "entities": {
                "city": "Sydney",
                "time_of_day": "evening"
            },
            "intent": "forecast_humidity"
        }
    },
    {
        "user_input": "Check the humidity forecast for Mumbai on the 20th.",
        "intent_extraction": {
            "entities": {
                "city": "Mumbai",
                "date": "2024-06-20"
            },
            "intent": "forecast_humidity"
        }
    },
    {
        "user_input": "Will the humidity be high in Rome on Saturday?",
        "intent_extraction": {
            "entities": {
                "city": "Rome",
                "date": "2024-06-17"
            },
            "intent": "forecast_humidity"
        }
    },
    {
        "user_input": "What's the humidity forecast in Athens for next week?",
        "intent_extraction": {
            "entities": {
                "city": "Athens",
                "date": "2024-06-19"
            },
            "intent": "forecast_humidity"
        }
    },
    {
        "user_input": "How humid will it be in Los Angeles on Tuesday?",
        "intent_extraction": {
            "entities": {
                "city": "Los Angeles",
                "date": "2024-06-18"
            },
            "intent": "forecast_humidity"
        }
    },
    {
        "user_input": "What's the wind speed like in New York right now?",
        "intent_extraction": {
            "entities": {
                "city": "New York"
            },
            "intent": "current_wind"
        }
    },
    {
        "user_input": "How fast is the wind blowing in Tokyo currently?",
        "intent_extraction": {
            "entities": {
                "city": "Tokyo"
            },
            "intent": "current_wind"
        }
    },
    {
        "user_input": "Can you tell me the current wind speed in Paris?",
        "intent_extraction": {
            "entities": {
                "city": "Paris"
            },
            "intent": "current_wind"
        }
    },
    {
        "user_input": "What's the wind speed in Berlin right now?",
        "intent_extraction": {
            "entities": {
                "city": "Berlin"
            },
            "intent": "current_wind"
        }
    },
    {
        "user_input": "Tell me the current wind speed in Sydney.",
        "intent_extraction": {
            "entities": {
                "city": "Sydney"
            },
            "intent": "current_wind"
        }
    },
    {
        "user_input": "How strong is the wind in Rome at the moment?",
        "intent_extraction": {
            "entities": {
                "city": "Rome"
            },
            "intent": "current_wind"
        }
    },
    {
        "user_input": "Check the current wind speed in London.",
        "intent_extraction": {
            "entities": {
                "city": "London"
            },
            "intent": "current_wind"
        }
    },
    {
        "user_input": "What's the wind speed like in Bangkok now?",
        "intent_extraction": {
            "entities": {
                "city": "Bangkok"
            },
            "intent": "current_wind"
        }
    },
    {
        "user_input": "Can you tell me the wind speed in Singapore right now?",
        "intent_extraction": {
            "entities": {
                "city": "Singapore"
            },
            "intent": "current_wind"
        }
    },
    {
        "user_input": "How fast is the wind blowing in Hong Kong?",
        "intent_extraction": {
            "entities": {
                "city": "Hong Kong"
            },
            "intent": "current_wind"
        }
    },
    {
        "user_input": "What's the wind speed in Cape Town right now?",
        "intent_extraction": {
            "entities": {
                "city": "Cape Town"
            },
            "intent": "current_wind"
        }
    },
    {
        "user_input": "Tell me about the current wind in Vienna.",
        "intent_extraction": {
            "entities": {
                "city": "Vienna"
            },
            "intent": "current_wind"
        }
    },
    {
        "user_input": "What's the wind speed in Athens today?",
        "intent_extraction": {
            "entities": {
                "city": "Athens"
            },
            "intent": "current_wind"
        }
    },
    {
        "user_input": "How strong is the wind in Chicago right now?",
        "intent_extraction": {
            "entities": {
                "city": "Chicago"
            },
            "intent": "current_wind"
        }
    },
    {
        "user_input": "What will be the wind speed in Paris tomorrow?",
        "intent_extraction": {
            "entities": {
                "city": "Paris",
                "date": "2024-06-15"
            },
            "intent": "forecast_wind"
        }
    },
    {
        "user_input": "Can you give me the wind forecast for Tokyo next week?",
        "intent_extraction": {
            "entities": {
                "city": "Tokyo",
                "date": "2024-06-19"
            },
            "intent": "forecast_wind"
        }
    },
    {
        "user_input": "What's the wind forecast in Berlin for the weekend?",
        "intent_extraction": {
            "entities": {
                "city": "Berlin",
                "date": "2024-06-17",
                "time_of_day": "weekend"
            },
            "intent": "forecast_wind"
        }
    },
    {
        "user_input": "Tell me the wind forecast in New York for tomorrow.",
        "intent_extraction": {
            "entities": {
                "city": "New York",
                "date": "2024-06-16"
            },
            "intent": "forecast_wind"
        }
    },
    {
        "user_input": "What will the wind speed be in Sydney this evening?",
        "intent_extraction": {
            "entities": {
                "city": "Sydney",
                "time_of_day": "evening"
            },
            "intent": "forecast_wind"
        }
    },
    {
        "user_input": "Check the wind forecast for Mumbai on the 20th.",
        "intent_extraction": {
            "entities": {
                "city": "Mumbai",
                "date": "2024-06-20"
            },
            "intent": "forecast_wind"
        }
    },
    {
        "user_input": "Will the wind be strong in Rome on Saturday?",
        "intent_extraction": {
            "entities": {
                "city": "Rome",
                "date": "2024-06-17"
            },
            "intent": "forecast_wind"
        }
    },
    {
        "user_input": "What's the wind forecast in Athens for next week?",
        "intent_extraction": {
            "entities": {
                "city": "Athens",
                "date": "2024-06-19"
            },
            "intent": "forecast_wind"
        }
    },
    {
        "user_input": "How strong will the wind be in Los Angeles on Tuesday?",
        "intent_extraction": {
            "entities": {
                "city": "Los Angeles",
                "date": "2024-06-18"
            },
            "intent": "forecast_wind"
        }
    },

    # Additional sentences for each intent
    # current_rain
    {"user_input": "Is it raining in San Francisco now?",
     "intent_extraction": {"entities": {"city": "San Francisco"}, "intent": "current_rain"}},
    {"user_input": "Tell me if it's raining in Berlin.",
     "intent_extraction": {"entities": {"city": "Berlin"}, "intent": "current_rain"}},
    {"user_input": "Is it raining at the moment in Dubai?",
     "intent_extraction": {"entities": {"city": "Dubai"}, "intent": "current_rain"}},
    {"user_input": "Check if it's raining in Sydney right now.",
     "intent_extraction": {"entities": {"city": "Sydney"}, "intent": "current_rain"}},
    {"user_input": "What's the rain status in Toronto now?",
     "intent_extraction": {"entities": {"city": "Toronto"}, "intent": "current_rain"}},
    {"user_input": "Is it raining outside in Rome?",
     "intent_extraction": {"entities": {"city": "Rome"}, "intent": "current_rain"}},
    {"user_input": "Is it raining currently in Madrid?",
     "intent_extraction": {"entities": {"city": "Madrid"}, "intent": "current_rain"}},
    {"user_input": "Can you tell if it's raining in Amsterdam?",
     "intent_extraction": {"entities": {"city": "Amsterdam"}, "intent": "current_rain"}},
    {"user_input": "Is it raining right now in Moscow?",
     "intent_extraction": {"entities": {"city": "Moscow"}, "intent": "current_rain"}},
    {"user_input": "Check if there's rain in Singapore now.",
     "intent_extraction": {"entities": {"city": "Singapore"}, "intent": "current_rain"}},
    {"user_input": "What's the current rain status in Bangkok?",
     "intent_extraction": {"entities": {"city": "Bangkok"}, "intent": "current_rain"}},
    {"user_input": "Is it raining at this moment in Hong Kong?",
     "intent_extraction": {"entities": {"city": "Hong Kong"}, "intent": "current_rain"}},
    {"user_input": "Is it raining right now in Cape Town?",
     "intent_extraction": {"entities": {"city": "Cape Town"}, "intent": "current_rain"}},
    {"user_input": "Is it raining in Athens currently?",
     "intent_extraction": {"entities": {"city": "Athens"}, "intent": "current_rain"}},
    {"user_input": "What's the rain situation in Vienna now?",
     "intent_extraction": {"entities": {"city": "Vienna"}, "intent": "current_rain"}},
    {"user_input": "Tell me if it's raining in Rio de Janeiro.",
     "intent_extraction": {"entities": {"city": "Rio de Janeiro"}, "intent": "current_rain"}},
    {"user_input": "Is it raining right now in Melbourne?",
     "intent_extraction": {"entities": {"city": "Melbourne"}, "intent": "current_rain"}},
    {"user_input": "Check if it's raining in Los Angeles at the moment.",
     "intent_extraction": {"entities": {"city": "Los Angeles"}, "intent": "current_rain"}},
    {"user_input": "Is it raining now in Chicago?",
     "intent_extraction": {"entities": {"city": "Chicago"}, "intent": "current_rain"}},
    
    # forecast_rain
    {"user_input": "Will it rain in Los Angeles tomorrow morning?",
     "intent_extraction": {"entities": {"city": "Los Angeles", "date": "2024-06-16", "time_of_day": "morning"}, "intent": "forecast_rain"}},
    {"user_input": "What is the rain forecast in Seattle for the weekend?",
     "intent_extraction": {"entities": {"city": "Seattle", "date": "2024-06-17", "time_of_day": "weekend"}, "intent": "forecast_rain"}},
    {"user_input": "Is there any rain expected in Miami next week?",
     "intent_extraction": {"entities": {"city": "Miami", "date": "2024-06-19", "time_of_day": "week"}, "intent": "forecast_rain"}},
    {"user_input": "Will it rain in Tokyo on Friday?",
     "intent_extraction": {"entities": {"city": "Tokyo", "date": "2024-06-16"}, "intent": "forecast_rain"}},
    {"user_input": "What's the rain forecast for Mumbai on the 20th?",
     "intent_extraction": {"entities": {"city": "Mumbai", "date": "2024-06-20"}, "intent": "forecast_rain"}},
    {"user_input": "Is rain predicted in Mexico City for tomorrow night?",
     "intent_extraction": {"entities": {"city": "Mexico City", "date": "2024-06-16", "time_of_day": "night"}, "intent": "forecast_rain"}},
    {"user_input": "Will there be rain in Sydney this evening?",
     "intent_extraction": {"entities": {"city": "Sydney", "time_of_day": "evening"}, "intent": "forecast_rain"}},
    {"user_input": "Can you check if it'll rain in Paris this afternoon?",
     "intent_extraction": {"entities": {"city": "Paris", "time_of_day": "afternoon"}, "intent": "forecast_rain"}},
    {"user_input": "Is rain expected in New York tomorrow?",
     "intent_extraction": {"entities": {"city": "New York", "date": "2024-06-16"}, "intent": "forecast_rain"}},
    {"user_input": "What does the rain forecast look like for London tomorrow morning?",
     "intent_extraction": {"entities": {"city": "London", "date": "2024-06-16", "time_of_day": "morning"}, "intent": "forecast_rain"}},
    {"user_input": "Will it rain in Berlin on Saturday?",
     "intent_extraction": {"entities": {"city": "Berlin", "date": "2024-06-17"}, "intent": "forecast_rain"}},
    {"user_input": "Is there a rain forecast for Moscow in the next few days?",
     "intent_extraction": {"entities": {"city": "Moscow", "date": "2024-06-18"}, "intent": "forecast_rain"}},
    {"user_input": "Will it rain in Bangkok on Sunday?",
     "intent_extraction": {"entities": {"city": "Bangkok", "date": "2024-06-18"}, "intent": "forecast_rain"}},
    {"user_input": "What's the forecast for rain in Athens next week?",
     "intent_extraction": {"entities": {"city": "Athens", "date": "2024-06-19"}, "intent": "forecast_rain"}},
    {"user_input": "Can you check the rain forecast for Cape Town on the 15th?",
     "intent_extraction": {"entities": {"city": "Cape Town", "date": "2024-06-15"}, "intent": "forecast_rain"}},
    {"user_input": "Is rain expected in Madrid in the coming days?",
     "intent_extraction": {"entities": {"city": "Madrid", "date": "2024-06-17"}, "intent": "forecast_rain"}},
    {"user_input": "What will the rain forecast be for Vienna tomorrow night?",
     "intent_extraction": {"entities": {"city": "Vienna", "date": "2024-06-16", "time_of_day": "night"}, "intent": "forecast_rain"}},
    {"user_input": "Will it rain in Toronto tomorrow afternoon?",
     "intent_extraction": {"entities": {"city": "Toronto", "date": "2024-06-16", "time_of_day": "afternoon"}, "intent": "forecast_rain"}},
    
    {
        "user_input": "Hello",
        "intent_extraction": {
            "intent": "greet",
            "response": [
                "Hello! How can I assist you today?",
                "Hi there! What can I do for you?",
                "Greetings! How may I help you?",
                "Hey! Need help with something?",
                "Hi! What do you need assistance with?"
            ]
        }
    },
    {
        "user_input": "Hi",
        "intent_extraction": {
            "intent": "greet",
            "response": [
                "Hi! How can I help you today?",
                "Hello! What can I assist you with?",
                "Hey there! What do you need?",
                "Hi! How can I assist you?",
                "Greetings! How can I be of service?"
            ]
        }
    },
    {
        "user_input": "Good morning",
        "intent_extraction": {
            "intent": "greet",
            "response": [
                "Good morning! How can I assist you today?",
                "Morning! What can I do for you?",
                "Good morning! How may I help you?",
                "Hello! How's your morning going? Need any help?",
                "Hi! Good morning! How can I assist?"
            ]
        }
    },
    {
        "user_input": "Goodbye",
        "intent_extraction": {
            "intent": "bye",
            "response": [
                "Goodbye! Have a great day!",
                "Bye! Take care!",
                "See you later! Stay safe!",
                "Goodbye! I'm here if you need me.",
                "Farewell! Let me know if you need anything else."
            ]
        }
    },
    {
        "user_input": "See you later",
        "intent_extraction": {
            "intent": "bye",
            "response": [
                "See you later! Have a good one!",
                "Bye! Talk to you soon!",
                "Goodbye! I'm here whenever you need me.",
                "Catch you later! Stay safe!",
                "Take care! See you soon."
            ]
        }
    },
    {
        "user_input": "What can you do?",
        "intent_extraction": {
            "intent": "about_bot",
            "response": [
                "I can provide weather updates, forecasts, and general information. How can I assist you?",
                "I'm here to help with weather-related queries. How can I assist?",
                "I can answer your questions about the weather and more. What do you need?",
                "I provide information on weather and forecasts. How can I help?",
                "I'm a weather assistant here to provide you with updates. What can I do for you?"
            ]
        }
    },
    {
        "user_input": "Tell me about yourself",
        "intent_extraction": {
            "intent": "about_bot",
            "response": [
                "I’m a weather assistant bot here to provide you with up-to-date weather information. What would you like to know?",
                "I'm a bot designed to help with weather information. How can I assist you?",
                "I was created to provide weather updates and forecasts. What can I do for you?",
                "I'm your go-to assistant for weather info. What would you like to ask?",
                "I help people with weather-related queries. How can I assist you today?"
            ]
        }
    },
    {
        "user_input": "Who made you?",
        "intent_extraction": {
            "intent": "about_bot",
            "response": [
                "I was created by a team of developers to help you with weather-related queries. How can I assist you today?",
                "A group of developers built me to provide weather information. How can I help?",
                "I was designed by developers to assist with weather updates. What do you need?",
                "A team of skilled developers created me to help you with weather info. How can I assist?",
                "I was made by developers to offer weather forecasts and updates. What can I do for you?"
            ]
        }
    },
    {
        "user_input": "What is your purpose?",
        "intent_extraction": {
            "intent": "about_bot",
            "response": [
                "My purpose is to assist you with weather updates and forecasts. How can I help you today?",
                "I'm here to provide weather information and answer your questions. What do you need?",
                "I help with weather-related queries and forecasts. How can I assist you?",
                "My job is to give you the latest weather updates. What would you like to know?",
                "I'm here to assist with anything related to weather. How can I help?"
            ]
        }
    },
    {
        "user_input": "Can you help me?",
        "intent_extraction": {
            "intent": "about_bot",
            "response": [
                "Of course! I'm here to help. What do you need assistance with?",
                "Yes, I can help! What would you like to know?",
                "Certainly! What can I assist you with?",
                "I'm here to help you. What do you need?",
                "Yes, I'm here to assist you. How can I help?"
            ]
        }
    }


]


In [67]:

# Load the dataset from Hugging Face
dataset = load_dataset("VatsalPatel18/Open-Weather-ChatBot-Intent")
intents = dataset['train']
print(intents)

Dataset({
    features: ['conversations'],
    num_rows: 689
})


In [68]:
from datasets import load_dataset, Dataset, concatenate_datasets
import json

# Step 1: Load the existing dataset
dataset = load_dataset("VatsalPatel18/Open-Weather-ChatBot-Intent")
intents = dataset['train']

# Step 2: Load the new intents from a JSON file
new_intents_data = new_intents

# Step 3: Convert the new intents into a Dataset format
# Convert the JSON data to a list of dictionaries with "text" and "intent" keys
# new_intents = [{"text": item["text"], "intent": item["intent"]} for item in new_intents_data]

# Create a new Dataset object from the list of dictionaries
new_intents_dataset = Dataset.from_list(new_intents_data)

# Step 4: Append the new intents to the existing dataset
combined_dataset = concatenate_datasets([intents, new_intents_dataset])

# Now, `combined_dataset` contains both the original and new intents.
print(combined_dataset)


Dataset({
    features: ['conversations', 'user_input', 'intent_extraction'],
    num_rows: 787
})


In [69]:

lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?', '!', '.', ',']

for example in intents['conversations'] + new_intents_data:
    pattern = example['user_input']
    intent = example['intent_extraction']['intent']
    print(intent)
    word_list = nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list, intent))
    if intent not in classes:
        classes.append(intent)

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))


forecast_weather
forecast_weather
forecast_weather
current_weather
forecast_weather
current_weather
current_weather
current_weather
current_weather
current_weather
current_weather
forecast_weather
current_weather
forecast_weather
current_weather
forecast_weather
current_weather
forecast_weather
forecast_weather
current_weather
current_weather
forecast_weather
forecast_weather
forecast_weather
forecast_weather
current_weather
forecast_weather
forecast_weather
current_weather
current_weather
forecast_weather
forecast_weather
current_weather
forecast_weather
current_weather
current_weather
current_weather
forecast_weather
forecast_weather
current_weather
current_weather
forecast_weather
forecast_weather
forecast_weather
current_weather
forecast_weather
forecast_weather
current_weather
forecast_weather
forecast_weather
forecast_weather
forecast_weather
forecast_weather
forecast_weather
forecast_weather
current_weather
current_weather
current_weather
forecast_weather
forecast_weather
foreca

In [70]:

training_data = []
# print(documents)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = [0] * len(classes)
    output_row[classes.index(doc[1])] = 1

    training_data.append([bag, output_row])

X_train = np.array([np.array(td[0]) for td in training_data])
y_train = np.array([np.array(td[1]) for td in training_data])


In [71]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

model = Sequential()

# Input layer
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(BatchNormalization())  # Normalize activations

# Hidden layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())  # Normalize activations

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())  # Normalize activations

# Output layer
model.add(Dense(len(y_train[0]), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')


Epoch 1/200
158/158 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.1663 - loss: 2.5743
Epoch 2/200
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6760 - loss: 1.2354  
Epoch 3/200
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7459 - loss: 0.9166
Epoch 4/200
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 998us/step - accuracy: 0.8040 - loss: 0.7101
Epoch 5/200
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8149 - loss: 0.6560
Epoch 6/200
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8122 - loss: 0.6189  
Epoch 7/200
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 994us/step - accuracy: 0.8387 - loss: 0.5267
Epoch 8/200
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7990 - loss: 0.6278  
Epoch 9/200
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8343 - loss: 0.5010  
Epoch 10/200
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 998us/step - accuracy: 0.8417 - loss: 0.4801
Epoch 11/200
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step - accuracy: 0.8335 - loss: 0.5162
Epoch 12/200
158/158 ━━━━

In [72]:

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]
    return return_list


In [73]:
import requests
import spacy

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

# Function to extract city name from message
def extract_city(message):
    doc = nlp(message)
    for entity in doc.ents:
        if entity.label_ == "GPE":  # GPE (Geopolitical Entity) includes cities
            return entity.text
    return None

# Function to get weather data
def get_weather(city, api_key, forecast=False):
    base_url = "http://api.openweathermap.org/data/2.5/"
    endpoint = "weather" if not forecast else "forecast"
    url = f"{base_url}{endpoint}?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    return response.json() if response.status_code == 200 else None

# Function to generate weather-related responses with natural language
def generate_weather_response(city, weather_data, forecast=False, measurement=None):
    if not weather_data:
        return "Sorry, I couldn't retrieve the weather data. Please try again later."
    
    if forecast:
        weather = weather_data['list'][0]
    else:
        weather = weather_data
    
    description = weather['weather'][0]['description']
    temp = weather['main']['temp']
    response = f"The current weather in {city} is {description}. "

    # Adding more natural temperature description
    if temp < 10:
        response += f"It's quite cold with a temperature of {temp:.1f} degrees Celsius."
    elif temp < 20:
        response += f"The temperature is a bit cool at {temp:.1f} degrees Celsius."
    else:
        response += f"It's fairly warm with a temperature of {temp:.1f} degrees Celsius."
    
    # Adding measurement-based descriptions
    if measurement:
        if measurement == 'rain':
            rain_amount = weather.get('rain', {}).get('1h', 0)
            if rain_amount > 0:
                response += f" There has been {rain_amount:.1f} millimeters of rain in the last hour."
            else:
                response += " There's no rain reported in the last hour."
        elif measurement == 'humidity':
            humidity = weather['main'].get('humidity', 'N/A')
            response += f" The humidity level is at {humidity}%."
        elif measurement == 'wind':
            wind_speed = weather['wind'].get('speed', 0)
            if wind_speed > 10:
                response += f" It's very windy with a speed of {wind_speed:.1f} meters per second."
            elif wind_speed > 5:
                response += f" There's a gentle breeze at {wind_speed:.1f} meters per second."
            else:
                response += f" The wind is quite calm at {wind_speed:.1f} meters per second."

    return response

# Updated get_response function
def get_response(ints, intents_json, message):
    tag = ints[0]['intent']
    api_key = "3ee7a6f464dc5e628e13ba142069fbb7"  # Replace with your actual API key
    
    for intent in intents_json['conversations'] + new_intents_data:
        if intent['intent_extraction']['intent'] == tag:
            forecast = tag in ['forecast_weather', 'forecast_rain', 'forecast_humidity', 'forecast_wind']
            
            if tag in ['current_weather', 'forecast_weather']:
                city = extract_city(message)
                if not city:
                    return "Please specify a city."
                weather_data = get_weather(city, api_key, forecast)
                return generate_weather_response(city, weather_data, forecast)
            elif tag in ['current_rain', 'forecast_rain']:
                city = extract_city(message)
                if not city:
                    return "Please specify a city."
                weather_data = get_weather(city, api_key, forecast)
                return generate_weather_response(city, weather_data, forecast, measurement='rain')
            elif tag in ['current_humidity', 'forecast_humidity']:
                city = extract_city(message)
                if not city:
                    return "Please specify a city."
                weather_data = get_weather(city, api_key, forecast)
                return generate_weather_response(city, weather_data, forecast, measurement='humidity')
            elif tag in ['current_wind', 'forecast_wind']:
                city = extract_city(message)
                if not city:
                    return "Please specify a city."
                weather_data = get_weather(city, api_key, forecast)
                return generate_weather_response(city, weather_data, forecast, measurement='wind')
            else:
                return random.choice(intent['intent_extraction']['response'])
    
    return "I'm not sure how to help with that."


In [76]:

print("Chatbot is ready to talk! (type 'quit' to stop)")
while True:
    message = input("You: ")
    if message.lower() == "quit":
        break

    print("User: ", message)
    ints = predict_class(message, model)
    # print(ints)
    res = get_response(ints, intents, message)
    print(f"Bot: {res}")


Chatbot is ready to talk! (type 'quit' to stop)
User:  what is the weather like in nairobi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Bot: The current weather in nairobi is overcast clouds. It's fairly warm with a temperature of 21.6 degrees Celsius.
User:  rain in addis ababa
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Bot: The current weather in addis ababa is moderate rain. It's fairly warm with a temperature of 20.0 degrees Celsius. There's no rain reported in the last hour.
User:  the rain in shhsds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Bot: Please specify a city.


In [75]:
# import requests
# import spacy

# # Load the SpaCy model
# nlp = spacy.load("en_core_web_sm")

# # Function to extract city name from message
# def extract_city(message):
#     doc = nlp(message)
#     for entity in doc.ents:
#         if entity.label_ == "GPE":  # GPE (Geopolitical Entity) includes cities
#             return entity.text
#     return None

# # Function to get weather data
# def get_weather(city, api_key, forecast=False):
#     base_url = "http://api.openweathermap.org/data/2.5/"
#     endpoint = "weather" if not forecast else "forecast"
#     url = f"{base_url}{endpoint}?q={city}&appid={api_key}&units=metric"
#     response = requests.get(url)
#     return response.json() if response.status_code == 200 else None

# # Function to generate weather-related responses
# def generate_weather_response(city, weather_data, forecast=False, measurement=None):
#     if not weather_data:
#         return "Sorry, I couldn't retrieve the weather data. Please try again later."
    
#     if forecast:
#         weather = weather_data['list'][0]
#     else:
#         weather = weather_data
    
#     description = weather['weather'][0]['description']
#     temp = weather['main']['temp']
    
#     response = ""
    
#     if measurement:
#         measurement_data = weather.get(measurement, {})
#         if measurement == 'rain':
#             rain_amount = measurement_data.get('1h', '0')
#             response += f" The rain measure in the last hour is {rain_amount} millimeters."
#         if measurement == 'humidity':
#             humidity = weather['main'].get('humidity', 'N/A')
#             response += f" The humidity is {humidity}%."
#         if measurement == 'wind':
#             wind_speed = weather['wind'].get('speed', 'N/A')
#             response += f" The wind speed is {wind_speed} m/s."
        
#         response += f"The general {'forecast' if forecast else 'current'} weather in {city} is {description} with a temperature of {temp}°C."
#     return response

# # Updated get_response function
# def get_response(ints, intents_json, message):
#     tag = ints[0]['intent']
#     api_key = "3ee7a6f464dc5e628e13ba142069fbb7"  # Replace with your actual API key
    

#     for intent in intents_json['conversations'] + new_intents_data:
#         if intent['intent_extraction']['intent'] == tag:
#             forecast = tag in ['forecast_weather', 'forecast_rain', 'forecast_humidity', 'forecast_wind']
            
#             if tag in ['current_weather', 'forecast_weather']:
#                 city = extract_city(message)
    
#                 if not city:
#                     return "Please specify a city."
#                 weather_data = get_weather(city, api_key, forecast)
#                 return generate_weather_response(city, weather_data, forecast)
#             elif tag in ['current_rain', 'forecast_rain']:
#                 city = extract_city(message)
    
#                 if not city:
#                     return "Please specify a city."
#                 weather_data = get_weather(city, api_key, forecast)
#                 return generate_weather_response(city, weather_data, forecast, measurement='rain')
#             elif tag in ['current_humidity', 'forecast_humidity']:
#                 city = extract_city(message)
    
#                 if not city:
#                     return "Please specify a city."
#                 weather_data = get_weather(city, api_key, forecast)
#                 return generate_weather_response(city, weather_data, forecast, measurement='humidity')
#             elif tag in ['current_wind', 'forecast_wind']:
#                 city = extract_city(message)
    
#                 if not city:
#                     return "Please specify a city."
#                 weather_data = get_weather(city, api_key, forecast)
#                 return generate_weather_response(city, weather_data, forecast, measurement='wind')
#             else:
#                 # print(intent)
#                 return random.choice(intent['intent_extraction']['response'])
    
#     return "I'm not sure how to help with that."
